# ML Pipeline Preparation

In this notebook, we prototype a ML pipeline to preprocess and classify the text message data.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database
- Prepare feature and target variables X and Y

In [117]:
# import libraries
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, roc_auc_score, fbeta_score, make_scorer

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer



import nltk
from nltk import WordNetLemmatizer as Lemmatizer
from nltk import word_tokenize
from nltk import PorterStemmer
nltk.download(['punkt','wordnet'])

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /Users/hchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hchen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
database_path = 'Disaster_Response.db'
table_name = 'labelled_messages'
engine = create_engine('sqlite:///'+database_path)
df = pd.read_sql_table(table_name, engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Understand the data

In [3]:
sum(df.original.isna())

16046

In [4]:
df.original.nunique()

9630

In [5]:
df.shape

(26216, 40)

In [6]:
sum(df.genre.isna())

0

In [7]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [8]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

In [9]:
# columns of df
columns = df.columns.values
columns[1:4], columns[4:]

(array(['message', 'original', 'genre'], dtype=object),
 array(['related', 'request', 'offer', 'aid_related', 'medical_help',
        'medical_products', 'search_and_rescue', 'security', 'military',
        'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
        'missing_people', 'refugees', 'death', 'other_aid',
        'infrastructure_related', 'transport', 'buildings', 'electricity',
        'tools', 'hospitals', 'shops', 'aid_centers',
        'other_infrastructure', 'weather_related', 'floods', 'storm',
        'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
       dtype=object))

In [17]:
# read off predictors X and targets y
X = df[columns[1:4:2]]
y = df[columns[4:]]

In [19]:
X

,message,genre
0,Weather update - a cold front from Cuba that c...,direct
1,Is the Hurricane over or is it not over,direct
2,Looking for someone but no name,direct
3,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,"says: west side of Haiti, rest of the country ...",direct
...,...,...
26211,The training demonstrated how to enhance micro...,news
26212,A suitable candidate has been selected and OCH...,news
26213,"Proshika, operating in Cox's Bazar municipalit...",news
26214,"Some 2,000 women protesting against the conduc...",news


In [20]:
y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
y.shape

(26216, 36)

In [22]:
np.mean(y, axis=0)

related                   0.773650
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

In [25]:
# the category weather_related is subdivided into these smaller categories

sum(y['other_weather'] | y['cold'] | y['earthquake'] | y['fire'] | y['storm'] | y['floods'] == y['weather_related'])

26216

### 2. A tokenization function to process the text messages

In [28]:
def tokenize(text, stemmize=False):
    """
    perform the following transformation of text
    lower
    tokenize
    lemmatize
    stemmize
    
    Input:
    text - string
    stemmize - boolean, if True, then stemmize the tokens. Default = False.
    
    Output:
    list consists of stemmed tokens from the input text
    """
    # lower, tokenize
    tokens = word_tokenize(text.lower())
    
    lemmatizer = Lemmatizer()
    stem_tokens = [lemmatizer.lemmatize(x) for x in tokens]
    if stemmize:
        stemmer = PorterStemmer()
        stem_tokens = [stemmer.stem(x) for x in stem_tokens]
    
    
    
    return stem_tokens

In [29]:
# Example
tokenize(df.message[0])

['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

In [13]:
tokenize(df.message[0], stemmize=True)

['weather',
 'updat',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pa',
 'over',
 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

Pipeline structure:

- features

    - tfidf vectorization from text message
    
        - extract the 'message' column
        - tokenization
        - tfidfvectorization
        
    - one-hot encodings from genre
        
        - extract the 'genre' column
        - encoding
        
- classifiers

- GridSearchCV
    

In [38]:
# vectorize the function tokenize
vect_tokenize = np.vectorize(tokenize)
# supped up to a transformer
class Tokenizer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array(list(map(tokenize, X)))

# customized transformer to get dummy variables
class GetDummies(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.get_dummies(X)

    

def pipeline_multioutput(clf):
    

    pipeline = Pipeline([
    
        ('features', FeatureUnion([
        
            ('vect', Pipeline([
            
                ('message', FunctionTransformer(lambda x: x[:, 0])),
                ('tfidfvect',TfidfVectorizer(tokenizer=tokenize, stop_words='english'))
            ])),
        
            ('encoding', Pipeline([
            
                ('genre', FunctionTransformer(lambda x: x[:,1])), 
                ('dummies',GetDummies())]))])),

        ('multiclf', MultiOutputClassifier(clf))])
    
    return pipeline

In [39]:
clf = RandomForestClassifier()
pipeline = pipeline_multioutput(clf)

### 4. Train pipeline

- Split data into train and test sets
- Train pipeline

In [40]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X.values,y.values, test_size=0.35, random_state=57)

In [53]:
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=101)

In [92]:
# train model
pipeline.fit(X_train,y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('vect',
                                                 Pipeline(steps=[('message',
                                                                  FunctionTransformer(func=<function pipeline_multioutput.<locals>.<lambda> at 0x7f85004ba550>)),
                                                                 ('tfidfvect',
                                                                  TfidfVectorizer(stop_words='english',
                                                                                  tokenizer=<function tokenize at 0x7f851b3714c0>))])),
                                                ('encoding',
                                                 Pipeline(steps=[('genre',
                                                                  FunctionTransformer(func=<function pipeline_multioutput.<locals>.<lambda> at 0x7f85004ba8b0>)),
                                                          

In [94]:
# prediction on cv set
y_pred_cv = pipeline.predict(X_cv)


In [97]:
# score report on cv set
labels = y.columns.values

def test_report(y_true, y_pred, labels):
    
    for i in range(y_true.shape[1]):
        print(f' Label "{labels[i]}": \n {classification_report(y_true[:,i], y_pred[:,i])}')
        
        
test_report(y_cv, y_pred, labels)        

 Label "related": 
               precision    recall  f1-score   support

           0       0.24      0.12      0.16      1086
           1       0.76      0.88      0.82      3470
           2       0.00      0.00      0.00        32

    accuracy                           0.70      4588
   macro avg       0.33      0.33      0.32      4588
weighted avg       0.63      0.70      0.65      4588

 Label "request": 
               precision    recall  f1-score   support

           0       0.82      0.89      0.86      3786
           1       0.15      0.09      0.11       802

    accuracy                           0.75      4588
   macro avg       0.49      0.49      0.48      4588
weighted avg       0.71      0.75      0.73      4588

 Label "offer": 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      4565
           1       0.00      0.00      0.00        23

    accuracy                           0.99      4588
   macro avg    

/Users/hchen/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hchen/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hchen/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hchen/anaconda3/lib/pyth

 Label "storm": 
               precision    recall  f1-score   support

           0       0.91      0.95      0.93      4163
           1       0.09      0.05      0.06       425

    accuracy                           0.87      4588
   macro avg       0.50      0.50      0.50      4588
weighted avg       0.83      0.87      0.85      4588

 Label "fire": 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      4543
           1       0.00      0.00      0.00        45

    accuracy                           0.99      4588
   macro avg       0.50      0.50      0.50      4588
weighted avg       0.98      0.99      0.99      4588

 Label "earthquake": 
               precision    recall  f1-score   support

           0       0.90      0.93      0.91      4140
           1       0.10      0.08      0.09       448

    accuracy                           0.84      4588
   macro avg       0.50      0.50      0.50      4588
weighted avg    

#### Define a multi-output f1 scorer for tuning.

In [106]:
def multilabel_fbeta(y_true, y_pred, beta=1):
    fbeta_scores = []
    for i in range(y_true.shape[1]):
    
        fbeta_scores.append(fbeta_score(y_true[:,i], y_pred[:,i], average='macro', beta=beta))
    
    return np.mean(fbeta_scores)

In [114]:
multilabel_fbeta(y_cv, y_pred_cv, beta=1)

0.6109459897799865

In [109]:
multi_fbeta = make_scorer(multilabel_fbeta)

### 5. Hyperparameter tuning.



In [123]:
parameters = {'features__vect__tfidfvect__use_idf':[True, False],
              'multiclf__estimator__n_estimators':[10, 25], 
              'multiclf__estimator__min_samples_split':[2, 6, 10]}

In [125]:
import warnings
warnings.filterwarnings('ignore')

# it seems there was an issue with parallel fitting. only work when n_jobs = 1
grid = GridSearchCV(pipeline, 
                      param_grid=parameters,
                      n_jobs = 1,
                      verbose = 2, scoring=multi_fbeta)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END features__vect__tfidfvect__use_idf=True, multiclf__estimator__min_samples_split=2, multiclf__estimator__n_estimators=10; total time=  22.4s
[CV] END features__vect__tfidfvect__use_idf=True, multiclf__estimator__min_samples_split=2, multiclf__estimator__n_estimators=10; total time=  22.4s
[CV] END features__vect__tfidfvect__use_idf=True, multiclf__estimator__min_samples_split=2, multiclf__estimator__n_estimators=10; total time=  21.9s
[CV] END features__vect__tfidfvect__use_idf=True, multiclf__estimator__min_samples_split=2, multiclf__estimator__n_estimators=10; total time=  22.2s
[CV] END features__vect__tfidfvect__use_idf=True, multiclf__estimator__min_samples_split=2, multiclf__estimator__n_estimators=10; total time=  22.2s
[CV] END features__vect__tfidfvect__use_idf=True, multiclf__estimator__min_samples_split=2, multiclf__estimator__n_estimators=25; total time=  47.9s
[CV] END features__vect__tfidfvect__use_idf=T

[CV] END features__vect__tfidfvect__use_idf=False, multiclf__estimator__min_samples_split=10, multiclf__estimator__n_estimators=25; total time=  32.9s
[CV] END features__vect__tfidfvect__use_idf=False, multiclf__estimator__min_samples_split=10, multiclf__estimator__n_estimators=25; total time=  33.2s
[CV] END features__vect__tfidfvect__use_idf=False, multiclf__estimator__min_samples_split=10, multiclf__estimator__n_estimators=25; total time=  33.3s
[CV] END features__vect__tfidfvect__use_idf=False, multiclf__estimator__min_samples_split=10, multiclf__estimator__n_estimators=25; total time=  33.5s
[CV] END features__vect__tfidfvect__use_idf=False, multiclf__estimator__min_samples_split=10, multiclf__estimator__n_estimators=25; total time=  33.4s


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('vect',
                                                                        Pipeline(steps=[('message',
                                                                                         FunctionTransformer(func=<function pipeline_multioutput.<locals>.<lambda> at 0x7f85004ba550>)),
                                                                                        ('tfidfvect',
                                                                                         TfidfVectorizer(stop_words='english',
                                                                                                         tokenizer=<function tokenize at 0x7f851b3714c0>))])),
                                                                       ('encoding',
                                                                        Pipeline(steps=[('genre',
             

In [127]:
best_clf = grid.best_estimator_

In [130]:
grid.cv_results_

{'mean_fit_time': array([20.68360238, 45.64470582, 15.93288045, 34.92068038, 14.77354298,
        30.92863169, 21.01944385, 48.37862868, 15.89854317, 34.15823078,
        14.66937122, 30.75806842]),
 'std_fit_time': array([0.18125408, 0.33887852, 0.15232827, 1.61819573, 0.26580353,
        0.17947077, 0.22842549, 1.59719455, 0.12298954, 0.47560177,
        0.08628649, 0.22845776]),
 'mean_score_time': array([1.54106545, 2.49418745, 1.57029705, 2.52165828, 1.54707975,
        2.48366876, 1.53032117, 2.50094357, 1.55626569, 2.49539723,
        1.56065817, 2.50663261]),
 'std_score_time': array([0.04317275, 0.03622274, 0.04483233, 0.05799151, 0.0209031 ,
        0.02947631, 0.02214636, 0.03327047, 0.02649952, 0.02152395,
        0.02882102, 0.02628038]),
 'param_features__vect__tfidfvect__use_idf': masked_array(data=[True, True, True, True, True, True, False, False,
                    False, False, False, False],
              mask=[False, False, False, False, False, False, False, False,

In [131]:
y_pred_cv = best_clf.predict(X_cv)


### 6. Further imporvement